## λ calculus

This is an appendix to upcoming book ["Introduction to Theoretical Computer Science"](http://introtcs.org), which is also available online as a Jupyter notebook in the  [boazbk/nandnotebooks](https://github.com/boazbk/nandnotebooks) on Github. You can also try the [live binder version](https://mybinder.org/v2/gh/boazbk/nandnotebooks/master?filepath=lambda.ipynb).

The λ calculus is discussed in  [Chapter 7: "Equivalent Models of Computation"](http://introtcs.org/public/lec_07_other_models.html)

[Click here](https://mybinder.org/v2/gh/boazbk/nandnotebooks/master?filepath=lambda.ipynb) for the live Binder version. (Service can sometimes be slow.)

This Python notebook provides a way to play with the lamdba calculus and  evaluate lambda expressions of the form `λvar1(exp1) λvar2(exp2) ...`. If you don't know Python you can safely ignore the Python code and skip below to where we actually talk about the λ calculus itself.

To better fit with python there are two main differences:

* Instead of writing `λvar.exp` we write `λvar(exp)`

* Instead of simply concatenating two expressions `exp1 exp2` we use the `*` operator and write `exp1 * exp2`. We can also use `exp1, exp2` if they are inside a function call or a variable binding parenthesis.

* To reduce an expression `exp`, use `exp.reduce()`

* Since Python does not allow us to override the default `0` and `1` we use `_0` for `λx(y(y))` and `_1` for `λx(y(x))`. 

## Initalization 

The above is all the code for implementing the λ calculus. We now add some convenient global variables: 
λa .... λz and a ... z for variables, and 0 and 1.

In [50]:
from lambdalib import *
Lambdaexp.lambdanames  = {}

In [51]:
initids(globals())

In [52]:
# testing...
λy(y)

λα.(α)

In [53]:
λ(a,b)(a)

λα.(λβ.(α))

In [54]:
setconstants(globals(),{"1" : λ(x,y)(x) , "0" : λ(x,y)(y)  })

In [55]:
# testing
λa(λz(a))

1

## λ calculus playground

We can now start playing with the λ calculus

If you want to use the λ character you can copy paste it from here:  λ

Let's start with the function λx,y.y, also known as 0 

In [56]:
λa(λb(b))

0

Our string representation recognizes that this is the 0 function and so "pretty prints" it. To see the underlying λ expression you can use `__repr__()`

In [57]:
λa(λb(b)).__repr__()

'λα.(λβ.(β))'

Let's check that `_0` and `_1` behave as expected

In [58]:
_1(a,b)

a

In [59]:
_0(a,b)

b

In [60]:
_1

1

In [61]:
_1(_0)

λα.(0)

In [62]:
_1.__repr__()

'λα.(λβ.(α))'

Here is an exercise:

__Question:__ Suppose that $F = \lambda f. (\lambda x. (f x)f)$, $1 = \lambda x.(\lambda y.x)$ and $0=\lambda x.(\lambda y.y)$.
What is $F \; 1\; 0$?

__a.__ $1$

__b.__ $0$

__c.__ $\lambda x.1$

__d.__ $\lambda x.0$

Let's evaluate the answer

In [63]:
F=λf(λx((f*x)*f))
F

λα.(λβ.(((α β) α)))

In [64]:
F(_1)

λα.(((1 α) 1))

In [65]:
F(_1,_0)

0

In [66]:
ID = λa(a)
register(globals(),"ID")

### Some useful functions

Let us now add some of the basic functions in the λ calculus

In [67]:
NIL= λf(_1)
PAIR =λx(λy(λf(f*x*y)))
ISEMPTY= λp(p *(λx(λy(_0))))
HEAD = λp(p(_1))
TAIL  =λp(p * _0)
IF = λ(a,b,c)(a * b * c)

register(globals(),"NIL", "PAIR")

And test them out

In [68]:
ISEMPTY(NIL)

1

In [69]:
IF(_0,a,b)

b

In [70]:
IF(_1,a,b)

a

In [71]:
P=PAIR(_0,_1)

In [72]:
HEAD(P)

0

In [73]:
TAIL(P)

1

We can make lists of bits as follows:

In [74]:
def makelist(*L):
    """Construct a λ list of _0's and _1's."""
    if not L: return NIL
    h = _1 if L[0]   else _0
    return PAIR(h,makelist(*L[1:]))

In [75]:
L=makelist(1,0,1)
L

λα.(((α 1) ((PAIR 0) ((PAIR 1) NIL))))

In [76]:
HEAD(L)

1

In [77]:
TAIL(L)

λα.(((α 0) ((PAIR 1) NIL)))

In [78]:
HEAD(TAIL(L))

0

In [79]:
HEAD(TAIL(TAIL(L)))

1

## Recursion

We now show how we can implement recursion in the λ calculus. We start by doing this in Python.
Let's try to define XOR in a recursive way and then avoid recursion

In [80]:
# XOR of 2 bits
def xor2(a,b): return 1-b if a else b

# XOR of a list - recursive definition
def xor(L): return xor2(L[0],xor(L[1:])) if L else 0

xor([1,0,0,1,1])

1

Now let's try to make a _non recursive_ definition, by replacing the recursive call with a call to `me` which is a function that is given as an extra argument:

In [81]:
def myxor(me,L): return 0 if not L else xor2(L[0],me(L[1:]))

The first idea is to try to implement `xor(L)` as `myxor(myxor,L)` but this will not work:

In [82]:
def xor(L): return myxor(myxor,L)

try: 
    xor([0,1,1])
except Exception as e:
    print(e)

myxor() missing 1 required positional argument: 'L'


The issue is that `myxor` takes _two_ arguments, while in `me` we only supply one.
Thus, we will modify `myxor` to `tempxor` where we replace the call `me(x)` with `me(me,x)`:

In [83]:
def tempxor(me,L): return myxor(lambda x: me(me,x),L)

Let's check this out:

In [84]:
def xor(L): return tempxor(tempxor,L)

xor([1,0,1,1])

1

This works!

Let's now generatlize this to any function. The `RECURSE` operator will take a function `f` that takes two arguments `me` and `x` and return a function `g`  where the calls to  `me` are replaced with calls to `g`

In [85]:
def RECURSE(f):
    def ftemp(me,x): return f(lambda x: me(me,x),x)
    return lambda x: ftemp(ftemp,x)

xor = RECURSE(myxor)

xor([1,1,0])

0

### The λ version

We now repeat the same arguments with the λ calculus:

In [86]:
# XOR of two bits
XOR2 = λ(a,b)(IF(a,IF(b,_0,_1),b))

# Recursive XOR with recursive calls replaced by m parameter
myXOR = λ(m,l)(IF(ISEMPTY(l),_0,XOR2(HEAD(l),m(TAIL(l)))))

# Recurse operator (aka Y combinator)
RECURSE = λf((λm(f(m*m)))(λm(f(m*m))))

# XOR function
XOR = RECURSE(myXOR)

Let's test this out:

In [87]:
XOR(PAIR(_1,NIL)) # List [1]

1

In [88]:
XOR(PAIR(_1,PAIR(_0,PAIR(_1,NIL)))) # List [1,0,1]

0

In [89]:
XOR(makelist(1,0,1))

0

In [90]:
XOR(makelist(1,0,0,1,1))

1